# 한국어 QnA데이터를 이용한 단순 챗봇

- 해당 노트북에서는 모델 다운로드, 토큰화, 학습(Fine-Tune)하는 과정을 설명합니다.

# 데이터
---
- 해당 데이터는 [AI-Hub](https://aihub.or.kr/aidata/85)에서 신청 후 다운 받을 수 있습니다.
- 데이터 셋은 소상공인 및 공공민원 등 10개분야에 대한 대화 데이터셋입니다.
- 데이터 셋에서 dialog.zip에 있는 A-I 파일을 이용해 데이터 셋을 구성했습니다.
- 사용한 데이터 셋의 발화는 총 79,940번이며, 각 대화마다 3~15번의 문장으로 구성되어있습니다.
- 데이터의 도메인 분야는 음식점, 의류, 학원, 소매점, 생활서비스, 숙박업, 관광여가오락, 부동산으로 구성되어있습니다.


데이터 예시는 다음과 같습니다.

In [1]:
import pandas as pd


data = pd.read_excel(r"{{enter your data path}}\D 소매점(14,949).xlsx")

In [2]:
data

,SPEAKER,SENTENCE,DOMAINID,DOMAIN,CATEGORY,SPEAKERID,SENTENCEID,MAIN,SUB,QA,QACNCT,MQ,SQ,UA,SA,개체명,용어사전,지식베이스
0,고객,삼겹살 1근에 얼마에요?,D,소매,정육점,1,1,가격 문의,NaN,Q,NaN,삼겹살 1근에 얼마에요?,NaN,NaN,NaN,"삼겹살, 1근",NaN,"삼겹살/부위, 1근/중량"
1,점원,만원입니다,D,소매,정육점,0,2,가격 문의,NaN,A,NaN,NaN,NaN,NaN,만원입니다,만원,NaN,만원/금액
2,고객,넷이 먹을건데 2근이면 되나요?,D,소매,정육점,1,3,0인분 용량 문의,NaN,Q,NaN,넷이 먹을 건데 2근이면 되나요?,NaN,NaN,NaN,"넷, 2근",NaN,"넷/인원, 2근/중량"
3,점원,네 충분하세요,D,소매,정육점,0,4,0인분 용량 문의,NaN,A,NaN,NaN,NaN,NaN,네 충분하세요,NaN,NaN,NaN
4,고객,그럼 2근주세요,D,소매,정육점,1,5,용량별 고기 주문,NaN,Q,NaN,그럼 2근 주세요,NaN,NaN,NaN,2근,NaN,2근/중량
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14944,고객,이 떡들 다 포장 되어있는거 맞죠?,D,소매,떡집,1,947,포장 상품 구매 확인,NaN,Q,NaN,이 떡들 다 포장 되어있는거 맞죠?,NaN,NaN,NaN,NaN,NaN,NaN
14945,고객,한 박스로도 포장 되는거죠?,D,소매,떡집,1,948,포장단위 문의,NaN,Q,NaN,한 박스로도 포장 되는거죠?,NaN,NaN,NaN,NaN,NaN,NaN
14946,고객,박스 말고 그냥 팩 단위로 포장해주실 수 있나요?,D,소매,떡집,1,949,포장단위 문의,NaN,Q,NaN,박스 말고 그냥 팩 단위로 포장해주실 수 있나요?,NaN,NaN,NaN,NaN,NaN,NaN
14947,고객,포장하는게 너무 커서 좀 적게 포장해주시면 좋을 것 같은데요?,D,소매,떡집,1,950,포장단위 문의,NaN,Q,NaN,포장하는게 너무 커서 좀 적게 포장해주시면 좋을 것 같은데요?,NaN,NaN,NaN,NaN,NaN,NaN


# 데이터 전처리

- 해당 데이터는 엑셀파일로 되어있으므로 학습에 필요한 데이터 셋으로 만들기 위해 전처리를 진행합니다.
- SENTENCE와 SPEAKERID 두 컬럼을 이용하여 전처리를 진행합니다.
- 데이터에는 하나의 질문이 두개 이상의 데이터로 구분되어있는 경우가 있기 때문에 해당 부분을 처리해줍니다.
- 또한, 소수의 데이터가 단순 숫자로 읽히기 때문에 해당 부분을 예외처리해줍니다.
- 해당 노트북에서는 진행과정을 보여드리기 위해 하나의 엑셀파일만으로 진행하겠습니다.
- 만약, 노트북을 실행시키시려면 ```# 여기``` 부분을 지우고 진행해주세요.

In [3]:
import os
from tqdm.notebook import tqdm


def preprocessing(BASE_PATH, SAVE_PATH):
    question_lists, answer_lists = [], []
    all_data = []
    temp_count = 0       # 여기
    for fn in os.listdir(BASE_PATH):
        if temp_count > 0: # 여기
            break          # 여기
        temp_count = 1      # 여기
        if os.path.splitext(fn)[-1] != '.xlsx':
            continue
        print(fn)
        data = pd.read_excel(os.path.join(BASE_PATH, fn))
        all_sentence, id_list = [], []
        for sentence, speaker_id in tqdm(zip(data["SENTENCE"], data["SPEAKERID"]), total=len(data)):
            all_sentence.append(str(sentence))
            id_list.append(speaker_id)

        questions, answers = [], []
        i = 0
        while i < len(all_sentence):
            end = i + 1
            _id = id_list[i]
            if end >= len(all_sentence):
                break
            while end < len(all_sentence) and id_list[end] == _id:
                end += 1
            if _id == 1:
                try:
                    questions.append(" ".join(all_sentence[i:end]) + "</s>")
                except:
                    print(i, end)
                    break
            else:
                answers.append(" ".join(all_sentence[i:end]) + "</s>")
            i = end
        min_length = min(len(questions), len(answers))
        question_lists.extend(questions[:min_length])
        answer_lists.extend(answers[:min_length])
        for i in range(min_length):
            all_data.append(questions[i])
            all_data.append(answers[i])

    with open(SAVE_PATH, 'wt', encoding='utf-8') as f:
        for line in all_data:
            f.write(line + "\n")

In [4]:
DATA_DIRECTORY = "{{Enter Your Data Directory Path}}"
SAVE_PATH = "{{Enter Your Save File Path}}"

preprocessing(DATA_DIRECTORY, SAVE_PATH)

A 음식점(15,726).xlsx


In [5]:
with open(SAVE_PATH, 'rt', encoding='utf-8') as f:
    data = f.readline()
data

'지금 배달되나요?</s>\n'

# 데이터 로더

- 위의 data를 불러올 데이터 로더 클래스를 선언 합니다.
- 데이터를 Q와 A를 이어줍니다.

In [6]:
import csv

import torch
from torch.utils.data import Dataset


class ChatBotDataset(Dataset):
    def __init__(self, data_path, tokenizer):
        self.conversation = {}
        with open(data_path, 'rt', encoding='utf-8') as f:
            data = f.read().split("\n")[:-1]
            for i in range(0, len(data), 2):
                temp_conversation = tokenizer(data[i]+data[i+1])
                for key in temp_conversation:
                    if key not in self.conversation:
                        self.conversation[key] = []
                    self.conversation[key].append(temp_conversation[key])

    def __len__(self):
        return len(self.conversation['input_ids'])

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.conversation.items()}

모델에 입력으로 넣어주기 위한 데이터 세트를 구성하기 위해 ```torch.utils.data.Dataset``` 객체를 import 합니다. 

```Dataset```객체를 이용하여 ```ConversationDataset```을 구현합니다. 

해당 Dataset(ConversationDataset)은 ```data_path```, ```tokenizer```를 입력으로 받습니다.

```data_path```로 부터 txt 데이터를 읽어 옵니다.

전처리 과정에서 Question Answer 순서로 전처리를 했기 때문에 ```data[i] + data[i+1]```로 데이터를 가져옵니다.

해당 데이터는 ```Q<eos>A<eos>``` 구조를 띄고 있습니다.


그렇게 생성된 문장을 ```tokenizer```를 이용하여 endocing 해 줍니다.
```python
temp_conversation = tokenizer(data[i]+data[i+1])
```

    
```__len__```과 ```__getitem__```을 구현하여 훈련할 모델에서 데이터를 불러 올 수 있게 설정해줍니다.


### 파라미터

- DATA_PATH : Fine-Tuning에 사용할 데이터 경로를 지정합니다. 본 노트북에선 앞서 전처리한 데이터를 이용합니다.
- MODEL_TYPE : 허깅페이스의 KoGPT-2를 설정합니다.

In [7]:
DATA_PATH = SAVE_PATH
MODEL_TYPE = "taeminlee/kogpt2"

데이터 세트를 위한 변수를 설정해 줍니다.

In [8]:
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast

tokenizer = PreTrainedTokenizerFast.from_pretrained(MODEL_TYPE)

저희는 SKT-AI에서 공개한 KoGPT2를 [허깅페이스](https://huggingface.co/taeminlee/kogpt2)에서 이용가능 하게 해주기 때문에 해당 모델을 이용하여 Fine-Tuning을 진행하겠습니다.

모델을 Fine-Tuning하기 위한 Tokenizer도 같이 사용하겠습니다.

In [9]:
dataset = ChatBotDataset(data_path=DATA_PATH, tokenizer=tokenizer)

## 디바이스 설정

In [10]:
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'

### 파라미터 설정

- BATCH_SIZE : 데이터 로더의 배치 사이즈를 설정합니다.
- EPOCHS : Fine-Tuning할 Epoch을 설정합니다.
- LEARNING_RATE : 모델 학습시, lr을 설정합니다.
- WARMUP_STEPS : 스케쥴러의 warmup을 진행할 step을 설정합니다.
- OUTPUT_FOLDER : 모델 저장 경로를 지정해 줍니다.

In [11]:
BATCH_SIZE = 1
EPOCHS = 3
LEARNING_RATE = 3e-5
WARMUP_STEPS = 100
OUTPUT_FOLDER = r"{{enter your save model path}}\models"

배치사이즈가 결정 되었으므로 ```torch.utils.data.DataLoader```를 사용하여 모델이 학습하기 위한 데이터를 load해주는 데이터 로더를 생성합니다.

# 데이터 로더

- 각 문장(input_ids)들의 길이가 서로 다르기 때문에, 배치 학습을 진행하지 못합니다.
- 이 불편함을 해결해주는게 DataLoader에서 제공하는 [collate_fn](https://pytorch.org/docs/stable/data.html)을 이용하면 됩니다.
- 배치 크기마다 패딩을 시켜주면 됩니다.
- 해당 토크나이저에서는 pad_token_id = 3 입니다.
- collate_fn에 넘겨줄 함수를 구현합니다.

In [12]:
def batch_padder(batch):
    max_length = -1
    pad_token_id = 3
    train_ids, attention_mask = [], []
    for data in batch:
        max_length = max(max_length, len(data['input_ids']))

    for i in range(len(batch)):
        train_ids.append(torch.cat([batch[i]["input_ids"],
                                           torch.LongTensor([pad_token_id] * (max_length - len(batch[i]["input_ids"])))]))
        attention_mask.append(torch.cat([batch[i]["attention_mask"],
                                           torch.LongTensor([0] * (max_length - len(batch[i]["attention_mask"])))]))
    return torch.stack(train_ids, 0), torch.stack(attention_mask, 0)

In [13]:
from torch.utils.data import Dataset, DataLoader


data_loader = DataLoader(dataset, batch_size=BATCH_SIZE,
                            collate_fn=batch_padder, shuffle=True)

In [14]:
for d in data_loader:
    print(d)
    break

(tensor([[28143, 47446, 19022,     1,   104, 25390,  3250, 43675, 47774,     1]]), tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]))


## Pre-trained MODEL LOAD
- 허깅페이스의 KoGPT2모델을 Load합니다.

In [15]:
from transformers import GPT2LMHeadModel


model = GPT2LMHeadModel.from_pretrained(MODEL_TYPE)

앞서 선언해둔 device와 함께 GPU사용이 가능하다면, ```model.to(device)```을 이용해 모델을 GPU로 보냅니다.

모델을 학습하기위해 Optimizer는 AdamW를 사용하고 scheduler를 선언 합니다.

In [16]:
from transformers import AdamW, get_linear_schedule_with_warmup


model = model.to(device)
model.train()
optimizier = AdamW(model.parameters(), lr=LEARNING_RATE)
scheduler = get_linear_schedule_with_warmup(optimizier, WARMUP_STEPS, len(data_loader) - WARMUP_STEPS, -1)

학습된 모델을 저장할 곳이 없을 경우 생성합니다.

In [17]:
import os


if not os.path.exists(OUTPUT_FOLDER):
    os.mkdir(OUTPUT_FOLDER)

## Fine-Tuning

모델 FineTuning 코드입니다.

노트북에서 보여드리기 위하여, 간단히 ```temp_count``` 변수를 설정하여 각 epoch당 5번의 배치만 간단히 학습하는 모습을 보여드리겠습니다.

실제로 실행하실경우 ```#여기```로 주석처리된 라인을 지우고 실행해주세요.

에폭당 변화를 보기위하여 total_loss와 total_count를 선언하고, data_loader를 이용하여 데이터를 가져옵니다.

모델과 데이터를 GPU로 보낸 후 학습을 진행합니다.

In [18]:
from tqdm.notebook import tqdm


def fine_tuning_runner(model, optim, data_loader, scheduler, epochs, save_path):
    model = model.to(device)
    model.train()
    print("=" * 15, "TRAIN MODEL", "=" * 15)
    temp_count = 0              # 여기
    for epoch in range(epochs):
        if temp_count > 0:      # 여기
            break               # 여기 
        print(f'EPOCH : {epoch}, started' + "=" * 30)
        total_loss = 0.0
        total_count = 0
        with tqdm(data_loader, desc="Train Epoch #{}".format(epoch)) as t:
            for train_ids, attention_masks in t:
                temp_count += 1        # 여기
                if temp_count > 5:     # 여기
                    model.save_pretrained(os.path.join(save_path, "temp"))  # 여기
                    break             # 여기
                train_ids, attention_masks = train_ids.to(device), attention_masks.to(device)
                outputs = model(train_ids, attention_mask=attention_masks, labels=train_ids)
                loss = outputs[0]

                total_loss += loss.detach().data
                total_count += 1
                t.set_postfix(loss='{:.6f}'.format(total_loss / total_count))
                optim.zero_grad()
                scheduler.optimizer.zero_grad()
                loss.backward()
                optim.step()
                scheduler.step()

        model.save_pretrained(os.path.join(save_path, f"epoch_{epoch}"))

In [19]:
fine_tuning_runner(model, optimizier, data_loader, scheduler, EPOCHS, OUTPUT_FOLDER)

=============== TRAIN MODEL ===============
EPOCH : 0, started==============================


## Sentence-Generate
- FINETUNE_MODEL_PATH : 학습된 모델의 경로를 설정합니다.

In [20]:
FINETUNE_MODEL_PATH = OUTPUT_FOLDER

QnA_Service_MODEL에 해당 가중치를 불러와 load 시켜 줍니다.


본 노트북에서는 진행과정을 보여드리기 위해 '/temp'에 저장되었습니다. 따라서 ```"/temp"``` 부분을 지워주세요.

In [21]:
QnA_Service_MODEL = GPT2LMHeadModel.from_pretrained(FINETUNE_MODEL_PATH+"/temp")

In [22]:
example_text = "3박4일 정도 놀러가고 싶다"
encoded_text = tokenizer.encode(example_text, add_special_tokens=True, return_tensors="pt")

In [23]:
encoded_text

tensor([[  141, 47650, 47514, 47471,  1057,  2211, 47593,  2999,  5314]])

In [24]:
generated_sentence = QnA_Service_MODEL.generate(encoded_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
C:\Users\loveg\Anaconda3\lib\site-packages\transformers\generation_utils.py:963: UserWarning: `max_length` is deprecated in this function, use `stopping_criteria=StoppingCriteriaList(MaxLengthCriteria(max_length=max_length))` instead.
  warnings.warn(


In [25]:
generated_sentence

tensor([[  141, 47650, 47514, 47471,  1057,  2211, 47593,  2999,  5314, 47654,
         47447,   317, 47440,     1,     0,   104,   533, 10469,   167,  3559]])

In [26]:
decoded_text = tokenizer.decode(generated_sentence[0], skip_special_tokens=True)

In [27]:
decoded_text

'3박4일 정도 놀러가고 싶다”고 말했다. 이 때문에 일각에서는 ‘박근혜'